# Introduction

We are an urban safety planner in Japan, and we are analyzing which areas of Japan need extra earthquake reinforcement.  Which areas are both high in population density and prone to earthquakes?

<center>
<img src="https://i.imgur.com/Kuh9gPj.png" width="450"><br/>
</center>

In [1]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex3 import *

We define a function `embed_map()` for displaying interactive maps. It accepts two arguments: the variable containing the map, and the name of the HTML file where the map will be saved.

This function ensures that the maps are visible [in all web browsers](https://github.com/python-visualization/folium/issues/812).

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Do earthquakes coincide with plate boundaries?

To answer this question, we create a DataFrame `plate_boundaries` that shows global plate boundaries.  The "coordinates" column is a list of (latitude, longitude) locations along the boundaries.

In [3]:
plate_boundaries = gpd.read_file("../input/geospatial-learn-course-data/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()


,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(-5.444200361999947, 133.6808931800001), (-5...."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(-7.760600482999962, 125.47879802900002), (-7..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(19.817899819000047, 120.09999798800004), (19..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(26.175899215000072, 143.20620700100005), (26..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(0.41880004000006466, 132.8273013480001), (0...."


Next, we load the historical earthquake data into a DataFrame `earthquakes`.

In [4]:
# Load the data and print the first 5 rows
earthquakes = pd.read_csv("../input/geospatial-learn-course-data/earthquakes1970-2014.csv", parse_dates=["DateTime"])
min_mag = earthquakes['Magnitude'].min()
min_mag
min_seism = earthquakes[earthquakes['Magnitude'] == 6.0]
min_seism



,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
4,1970-01-16 08:05:39.000,60.2800,-152.6600,85.00,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN
12,1970-02-24 02:07:36.800,30.5760,103.0310,33.00,6.0,Ms,94.0,NaN,NaN,0.00,NEI,1.970022e+09
13,1970-02-27 07:07:56.500,50.1300,-179.5898,7.00,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN
17,1970-03-10 04:58:26.200,44.7980,148.8700,40.00,6.0,Mb,65.0,NaN,NaN,0.00,NEI,1.970031e+09
19,1970-03-15 12:39:17.800,-29.6530,-69.4990,119.00,6.0,Mb,118.0,NaN,NaN,0.00,NEI,1.970032e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
5284,2013-10-29 10:37:55.930,-61.6945,154.7302,9.97,6.0,Mw,NaN,96.0,8.0,0.94,us,2.013103e+09
5288,2013-11-02 15:52:46.140,-23.6305,-112.5912,9.96,6.0,Mw,NaN,35.0,5.0,0.82,us,2.013110e+09
5294,2013-11-19 13:32:53.730,2.6250,128.4462,59.77,6.0,Mw,NaN,18.0,2.0,0.86,us,2.013112e+09
5295,2013-11-19 17:00:44.160,18.4753,145.2041,511.00,6.0,Mw,NaN,10.0,2.0,1.05,us,2.013112e+09


The code cell below visualizes the plate boundaries on a map. We use all of the earthquake data to add a heatmap to the same map, to determine whether earthquakes coincide with plate boundaries.  

In [5]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)
HeatMap(data = earthquakes[['Latitude', 'Longitude']], radius=15).add_to(m_1)
# Show the map
embed_map(m_1, 'q_1.html')

So, given the map above, do earthquakes coincide with plate boundaries?

Yes, eathquakes coincide with plate boundaries

### 2) Is there a relationship between earthquake depth and proximity to a plate boundary in Japan?

We recently read that the depth of earthquakes tells us [important information](https://www.usgs.gov/faqs/what-depth-do-earthquakes-occur-what-significance-depth?qt-news_science_products=0#qt-news_science_products) about the structure of the earth.  We are interested in seeing if there are any intereresting global patterns, and we'd also like to understand how depth varies in Japan.



In [6]:
# we Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
# We add a map to visualize earthquake depth
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

#  we Add a bubble map to the base map
for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        radius=2000,
        color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_2)

# View the map
embed_map(m_2, 'q_2.html')

Can we detect a relationship between proximity to a plate boundary and earthquake depth?  Does this pattern hold globally? In Japan?

In the northern half of Japan, it does appear that earthquakes closer to plate boundaries tend to be shallower (and earthquakes farther from plate boundaries are deeper). This pattern is repeated in other locations, such as the western coast of South America. But, it does not hold everywhere (for instance, in China, Mongolia, and Russia).

### 3) Which prefectures have high population density?

We run the next code cell (without changes) to create a GeoDataFrame `prefectures` that contains the geographical boundaries of Japanese prefectures.

In [7]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("../input/geospatial-learn-course-data/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.65330, 137.09546 ..."
Akita,"MULTIPOLYGON (((139.55725 39.20330, 139.55765 ..."
Aomori,"MULTIPOLYGON (((141.39860 40.92472, 141.39806 ..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


The next code cell creates a DataFrame `stats` containing the population, area (in square kilometers), and population density (per square kilometer) for each Japanese prefecture.  Run the code cell without changes.

In [8]:
# DataFrame containing population of each prefecture
population = pd.read_csv("../input/geospatial-learn-course-data/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# We calculate area (in square kilometers) of each prefecture and add density (per square km) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


Use the next code cell to create a choropleth map to visualize population density.

In [9]:
# We Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# We create a choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__, 
           data= stats['density'], 
           key_on= "feature.id", 
           fill_color='YlGnBu', 
           legend_name='Population density in Japan (per square kilometer)',
          ).add_to(m_3)

# View the map
embed_map(m_3, 'q_3.html')

Which three prefectures have relatively higher density than the others?  Are they spread throughout the country, or all located in roughly the same geographical region?  (*In case we are unfamiliar with Japanese geography, we might find [this map](https://en.wikipedia.org/wiki/Prefectures_of_Japan) useful to answer the questions.)*

Tokyo, Kanagawa, and Osaka have the highest population density. All of these prefectures are located in central Japan, and Tokyo and Kanagawa are adjacent.

### 4) Which high-density prefecture is prone to high-magnitude earthquakes?

We create a map to suggest one prefecture that might benefit from earthquake reinforcement.  The map should visualize both density and earthquake magnitude.

In [10]:
# Create a base map
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# map creation using choropleth
Choropleth(geo_data=prefectures['geometry'].__geo_interface__, 
           data= stats['density'], 
           key_on= "feature.id", 
           fill_color='YlGnBu', 
           legend_name = "Population density per square kilometer"
          ).add_to(m_4)

def color_producer(magnitude):
    if magnitude > 6.5:
        return 'red'
    else:
        return 'green'

# Add a bubble map to the base map
for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)


# View the map
embed_map(m_4, 'q_4.html')

Which prefecture do you recommend for extra earthquake reinforcement?

While there's no clear, single answer to this question, there are a few reasonable options. Tokyo is by far the most densely populated prefecture and has also experienced a number of earthquakes. Osaka is relatively less densely populated, but experienced an earthquake that was relatively stronger than those near Tokyo. And, the long coast of Kanagawa (in addition to its high density and the historical proximity of strong earthquakes) might lead us to worry about the added potential tsunami risk.